In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [4]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [157]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        text = ' '.join([tag.get_text(strip=False).replace('\r', ' ').replace('\n', ' ').strip() for tag in i])
        cleaned_text = re.sub(r'\s+', ' ', text).strip()
        part_list.append(cleaned_text)
        part_text = '\n'.join(part_list)
    return part_text
        

In [187]:
link = "https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx"


In [192]:
def craw_thongtu(link): 
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')
    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', ' ').replace('\n', ' ').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', ' ').replace('\n', ' ').strip()
            if 'ngày' in text or 'Ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]
    all_content = content1.find_all('p')

    pattern = pattern = r"Điều\s\d+\s*\."
    indexes = []
    # Duyệt qua các thẻ <p> và tìm những thẻ khớp với pattern
    for idx, tag in enumerate(all_content):
        if re.search(pattern, tag.get_text()):  # Kiểm tra nội dung văn bản của thẻ <p>
            indexes.append(idx)


    pattern_chuong = r"Chương\s[IVX]+"
    indexes_chuong = []
    # Duyệt qua các thẻ <p> và tìm những thẻ khớp với pattern
    for idx, tag in enumerate(all_content):
        if re.match(pattern_chuong, tag.get_text()):  # Kiểm tra nội dung văn bản của thẻ <p>
            indexes_chuong.append(idx)

    final_index = sorted(indexes + indexes_chuong)
    end_pattern = r"Nơi nhận\s*:\s*.*?\s*Thủ tướng\s*,\s*"
    # Duyệt qua các thẻ <p> và tìm những thẻ khớp với pattern
    for idx, tags in enumerate(all_content):
        text = ''.join([tag.get_text(strip=False).replace('\r', ' ').replace('\n', ' ') for tag in tags])
        cleaned_text = re.sub(r'\s+', ' ', text).strip()
        if re.search(end_pattern,cleaned_text):
            end_idx = idx
            final_index.append(end_idx)
    # Lọc danh sách để loại bỏ các giá trị lớn hơn idx

    final_index = [value for value in final_index if value <= end_idx]

    for a, b in zip(final_index, final_index[1:]):
        if a in indexes_chuong:
            True
        else:
            print(f"Từ {a} đến {b}")
            print(part_text(all_content,a,b))
            info_list.append(part_text(all_content,a,b))
    print(info_list)
            
    return info_list
            

In [193]:
craw_thongtu(link)

https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx
Từ 14 đến 16
Điều 1. Phạm vi điều chỉnh
Thông tư này quy định Quy tắc xuất xứ hàng hóa trong Hiệp định Thương mại tự do giữa Việt Nam và Liên minh châu Âu (EVFTA).
Từ 16 đến 21
Điều 2. Đối tượng áp dụng
Thông tư này áp dụng đối với:
1. Cơ quan, tổ chức cấp Giấy chứng nhận xuất xứ hàng hóa (C/O).
2. Thương nhân.
3. Cơ quan, tổ chức, cá nhân liên quan đến xuất xứ hàng hóa xuất khẩu và nhập khẩu.
Từ 21 đến 41
Điều 3. Giải thích từ ngữ
Theo quy định tại Thông tư này, các thuật ngữ dưới đây được hiểu như sau:
1. “Chương”, “Nhóm” và “Phân nhóm” là các Chương (2 số), Nhóm (4 số) và Phân nhóm (6 số) sử dụng trong Hệ thống hài hòa mô tả và mã hóa hàng hóa (sau đây gọi là “Hệ thống Hài hòa” hay “HS”).
2. “Được phân loại” là sự phân loại hàng hóa hoặc nguyên liệu theo Chương, Nhóm hay Phân nhóm cụ thể của Hệ thống Hài hòa.
3. “Lô hàng” là các sản phẩ

['https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx',
 '11/2020/TT-BCT',
 '15-06-2020',
 'Điều 1. Phạm vi điều chỉnh\nThông tư này quy định Quy tắc xuất xứ hàng hóa trong Hiệp định Thương mại tự do giữa Việt Nam và Liên minh châu Âu (EVFTA).',
 'Điều 2. Đối tượng áp dụng\nThông tư này áp dụng đối với:\n1. Cơ quan, tổ chức cấp Giấy chứng nhận xuất xứ hàng hóa (C/O).\n2. Thương nhân.\n3. Cơ quan, tổ chức, cá nhân liên quan đến xuất xứ hàng hóa xuất khẩu và nhập khẩu.',
 'Điều 3. Giải thích từ ngữ\nTheo quy định tại Thông tư này, các thuật ngữ dưới đây được hiểu như sau:\n1. “Chương”, “Nhóm” và “Phân nhóm” là các Chương (2 số), Nhóm (4 số) và Phân nhóm (6 số) sử dụng trong Hệ thống hài hòa mô tả và mã hóa hàng hóa (sau đây gọi là “Hệ thống Hài hòa” hay “HS”).\n2. “Được phân loại” là sự phân loại hàng hóa hoặc nguyên liệu theo Chương, Nhóm hay Phân nhóm cụ thể của Hệ thống Hài hòa.\n3. “Lô hà

In [194]:
all_link_info = []
for link in hyperlinks:
    try:
        info_list = craw_thongtu(link)
        all_link_info.append(info_list)
        # write_data_to_excel(phanloai_sheet,info_list)
    except:
        print(link + "lỗi")
        info_list = [link,"lỗi"]
        all_link_info.append(info_list)
        # write_data_to_excel(phanloai_sheet,info_list)

https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx
Từ 14 đến 16
Điều 1. Phạm vi điều chỉnh
Thông tư này quy định Quy tắc xuất xứ hàng hóa trong Hiệp định Thương mại tự do giữa Việt Nam và Liên minh châu Âu (EVFTA).
Từ 16 đến 21
Điều 2. Đối tượng áp dụng
Thông tư này áp dụng đối với:
1. Cơ quan, tổ chức cấp Giấy chứng nhận xuất xứ hàng hóa (C/O).
2. Thương nhân.
3. Cơ quan, tổ chức, cá nhân liên quan đến xuất xứ hàng hóa xuất khẩu và nhập khẩu.
Từ 21 đến 41
Điều 3. Giải thích từ ngữ
Theo quy định tại Thông tư này, các thuật ngữ dưới đây được hiểu như sau:
1. “Chương”, “Nhóm” và “Phân nhóm” là các Chương (2 số), Nhóm (4 số) và Phân nhóm (6 số) sử dụng trong Hệ thống hài hòa mô tả và mã hóa hàng hóa (sau đây gọi là “Hệ thống Hài hòa” hay “HS”).
2. “Được phân loại” là sự phân loại hàng hóa hoặc nguyên liệu theo Chương, Nhóm hay Phân nhóm cụ thể của Hệ thống Hài hòa.
3. “Lô hàng” là các sản phẩ

In [195]:
all_link_info

[['https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx',
  '11/2020/TT-BCT',
  '15-06-2020',
  'Điều 1. Phạm vi điều chỉnh\nThông tư này quy định Quy tắc xuất xứ hàng hóa trong Hiệp định Thương mại tự do giữa Việt Nam và Liên minh châu Âu (EVFTA).',
  'Điều 2. Đối tượng áp dụng\nThông tư này áp dụng đối với:\n1. Cơ quan, tổ chức cấp Giấy chứng nhận xuất xứ hàng hóa (C/O).\n2. Thương nhân.\n3. Cơ quan, tổ chức, cá nhân liên quan đến xuất xứ hàng hóa xuất khẩu và nhập khẩu.',
  'Điều 3. Giải thích từ ngữ\nTheo quy định tại Thông tư này, các thuật ngữ dưới đây được hiểu như sau:\n1. “Chương”, “Nhóm” và “Phân nhóm” là các Chương (2 số), Nhóm (4 số) và Phân nhóm (6 số) sử dụng trong Hệ thống hài hòa mô tả và mã hóa hàng hóa (sau đây gọi là “Hệ thống Hài hòa” hay “HS”).\n2. “Được phân loại” là sự phân loại hàng hóa hoặc nguyên liệu theo Chương, Nhóm hay Phân nhóm cụ thể của Hệ thống Hài hòa.\n3. 

In [181]:

# Mở file Excel
file_path = os.path.join(current_directory, 'FTA Template - Copy - Copy.xlsx')
workbook = openpyxl.load_workbook(file_path)


# Chọn worksheet
sheet = workbook["CHI TIẾT FTA "]  # Hoặc sử dụng workbook['SheetName'] để chọn một sheet cụ thể

# Danh sách để lưu đường link
hyperlinks = []

# Duyệt qua tất cả các ô trong worksheet
for row in sheet.iter_rows(min_row=4, max_row=4):
    for cell in row:
        # Kiểm tra xem ô có chứa hyperlink hay không
        if cell.hyperlink:
            # Lấy đường link và lưu vào danh sách
            hyperlinks.append(cell.hyperlink.target)

# In ra danh sách đường link
print(hyperlinks)


['https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-11-2020-TT-BCT-quy-dinh-Quy-tac-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-445632.aspx', 'https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-22-2016-TT-BCT-thuc-hien-Quy-tac-xuat-xu-hang-hoa-Hiep-dinh-Thuong-mai-hang-hoa-ASEAN-325243.aspx', 'https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-15-2010-TT-BCT-thuc-hien-quy-tac-xuat-xu-trong-Hiep-dinh-Thuong-mai-hang-hoa-ASEAN-An-Do-104363.aspx', 'https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-37-2022-TT-BCT-Quy-tac-xuat-xu-hang-hoa-Hiep-dinh-Doi-tac-kinh-te-ASEAN-Nhat-Ban-549088.aspx', 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-tu-12-2019-TT-BCT-xuat-xu-hang-hoa-trong-Hiep-dinh-khung-ve-hop-tac-kinh-te-toan-dien-420674.aspx', 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-tu-21-2019-TT-BCT-xuat-xu-hang-hoa-trong-Hiep-dinh-Thuong-mai-tu-do-ASEAN-Hong-Cong-427796.aspx', 'https://thuvienphapluat.vn/van-ban/Thuong-mai/Thong-tu-20-2014-TT-BC

In [182]:
print(workbook.sheetnames)

['CHI TIẾT FTA ', 'Sheet1', 'Sheet2']


In [184]:
output_sheet = workbook["Sheet2"]  # Lấy sheet mặc định

# Ghi từng dòng của danh sách vào các ô Excel
for row in all_link_info:
    output_sheet.append(row)  # Append từng list con vào sheet

In [196]:

# Mở file Excel
file_path = os.path.join(current_directory, 'FTA Template - Copy - Copy.xlsx')
workbook = openpyxl.load_workbook(file_path)

# Kiểm tra nếu "Sheet2" tồn tại, nếu không thì tạo mới
if "Sheet2" in workbook.sheetnames:
    output_sheet = workbook["Sheet2"]
else:
    output_sheet = workbook.create_sheet("Sheet2")

# Ghi từng dòng của danh sách vào các ô Excel
for row in all_link_info:
    output_sheet.append(row)  # Ghi từng list con vào sheet

# Lưu lại workbook sau khi ghi dữ liệu
workbook.save(file_path)
